# Resultados Finales y Visualizaciones - Reporte Técnico UPME

## Objetivos:
1.  Comparar la cobertura de edificios entre Google Open Buildings y Microsoft Building Footprints.
2.  Identificar los municipios PDET con mayor potencial de área de techo para energía solar.
3.  Generar mapas de calor y tablas resumen.

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from pathlib import Path

# Agregar directorio raíz al path para importar src
project_root = Path('..').resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.database.connection import get_database, load_config

# Configuración de estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Conexión a Base de Datos y Carga de Datos

In [ ]:
try:
    db = get_database()
    print(f"Conectado a base de datos: {db.name}")
    # Verificar si existe la colección pre-agregada
    if 'buildings_by_municipality' in db.list_collection_names():
        print("Colección 'buildings_by_municipality' encontrada. Usando pipeline de agregación.")
        use_aggregation = True
    else:
        print("Colección 'buildings_by_municipality' NO encontrada. Intentando cargar CSV pre-generado.")
        use_aggregation = False
except Exception as e:
    print(f"Error conectando a la base de datos: {e}")
    use_aggregation = False

## 2. Procesamiento de Datos (MongoDB Aggregation)
Se utiliza un pipeline de agregación de MongoDB para calcular densidades y métricas comparativas directamente en el servidor.

In [ ]:
if use_aggregation:
    buildings_coll = db.buildings_by_municipality
    
    # Pipeline copiado de deliverables/deliverable_4/scripts/02_generate_statistics.py
    pipeline = [
        {
            '$addFields': {
                'ms_density': {
                    '$cond': [{'$gt': ['$area_km2', 0]}, {'$divide': ['$microsoft.count', '$area_km2']}, 0]
                },
                'gg_density': {
                    '$cond': [{'$gt': ['$area_km2', 0]}, {'$divide': ['$google.count', '$area_km2']}, 0]
                },
                'ms_coverage': {
                    '$cond': [{'$gt': ['$area_km2', 0]}, {'$multiply': [{'$divide': ['$microsoft.total_area_km2', '$area_km2']}, 100]}, 0]
                },
                'gg_coverage': {
                    '$cond': [{'$gt': ['$area_km2', 0]}, {'$multiply': [{'$divide': ['$google.total_area_km2', '$area_km2']}, 100]}, 0]
                },
                'diff_count': {'$subtract': ['$google.count', '$microsoft.count']},
                'diff_pct': {
                    '$cond': [
                        {'$gt': ['$microsoft.count', 0]},
                        {'$multiply': [{'$divide': [{'$subtract': ['$google.count', '$microsoft.count']}, '$microsoft.count']}, 100]},
                        0
                    ]
                }
            }
        },
        {
            '$project': {
                '_id': 0,
                'muni_code': 1, 'muni_name': 1, 'dept_name': 1, 'pdet_region': 1,
                'area_municipal_km2': {'$round': ['$area_km2', 2]},
                'ms_buildings_count': '$microsoft.count',
                'ms_total_roof_area_km2': {'$round': ['$microsoft.total_area_km2', 4]},
                'ms_density_buildings_km2': {'$round': ['$ms_density', 2]},
                'gg_buildings_count': '$google.count',
                'gg_total_roof_area_km2': {'$round': ['$google.total_area_km2', 4]},
                'gg_density_buildings_km2': {'$round': ['$gg_density', 2]},
                'diff_count': 1,
                'diff_pct': {'$round': ['$diff_pct', 2]}
            }
        },
        {'$sort': {'pdet_region': 1, 'muni_name': 1}}
    ]
    
    print("Ejecutando agregación...")
    results = list(buildings_coll.aggregate(pipeline))
    df = pd.DataFrame(results)
    print(f"Datos cargados: {len(df)} municipios")
else:
    # Cargar CSV si la DB no está disponible o no tiene la colección
    csv_path = Path('../deliverables/deliverable_4/outputs/tables/municipalities_stats.csv')
    if csv_path.exists():
        print(f"Cargando datos desde CSV: {csv_path}")
        df = pd.read_csv(csv_path)
    else:
        print("No se encontraron datos. Por favor ejecute los scripts de la Entrega 4 primero.")
        df = pd.DataFrame()

df.head()

## 3. Visualizaciones Comparativas

In [ ]:
if not df.empty:
    # 1. Comparación de Conteos Totales
    total_ms = df['ms_buildings_count'].sum()
    total_gg = df['gg_buildings_count'].sum()
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=['Microsoft', 'Google'], y=[total_ms, total_gg], palette=['#00a4ef', '#4285f4'])
    plt.title('Total de Edificaciones Detectadas en Municipios PDET', fontsize=16)
    plt.ylabel('Número de Edificaciones (Millones)', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Añadir etiquetas de valor
    for i, v in enumerate([total_ms, total_gg]):
        plt.text(i, v, f'{v:,.0f}', ha='center', va='bottom', fontsize=12, fontweight='bold')
        
    plt.show()

In [ ]:
if not df.empty:
    # 2. Scatter Plot: Correlación de Conteos
    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=df, x='ms_buildings_count', y='gg_buildings_count', alpha=0.6, hue='pdet_region')
    
    # Línea de identidad (x=y)
    max_val = max(df['ms_buildings_count'].max(), df['gg_buildings_count'].max())
    plt.plot([0, max_val], [0, max_val], 'r--', label='Identidad (x=y)')
    
    plt.title('Correlación de Conteos: Microsoft vs Google', fontsize=16)
    plt.xlabel('Conteo Microsoft', fontsize=12)
    plt.ylabel('Conteo Google', fontsize=12)
    plt.legend(title='Región PDET', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
if not df.empty:
    # 3. Top 10 Municipios por Potencial Solar (Área de Techo - Google)
    top_10 = df.nlargest(10, 'gg_total_roof_area_km2')
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=top_10, x='gg_total_roof_area_km2', y='muni_name', palette='viridis')
    plt.title('Top 10 Municipios por Área Total de Techo (Google)', fontsize=16)
    plt.xlabel('Área Total de Techo (km²)', fontsize=12)
    plt.ylabel('Municipio', fontsize=12)
    plt.show()

In [ ]:
if not df.empty:
    # Tabla Resumen para el Reporte
    summary_table = top_10[['muni_name', 'dept_name', 'gg_buildings_count', 'gg_total_roof_area_km2']].copy()
    summary_table.columns = ['Municipio', 'Departamento', 'Edificaciones (Google)', 'Área Techo (km²)']
    print("Tabla para el reporte:")
    display(summary_table)